## From Zero Instructions:

1. Install Okareo's Python SDK: &nbsp;&nbsp;  `pip install okareo`  &nbsp;&nbsp;  (just run the cell below)

2. Get your API token from [https://app.okareo.com/](https://app.okareo.com/).  
   (Note: You will need to register first.)

3. Go directly to the **"2. Create your API Token"** link on the landing page. You can skip all other steps.

4. Set the environment variable `OKAREO_API_KEY` to your generated API token.

In [ ]:
%pip install okareo 

In [ ]:
%pip install chromadb

In [ ]:
%pip install pandas

In [1]:
# Webbizz is an example web business. 
# We load 10 short summaries about different business aspects to the vector database.
import chromadb
import pandas as pd    

chroma_client = chromadb.Client()

collection = chroma_client.create_collection(name="retrieval_test", metadata={"hnsw:space": "cosine"})
jsonObj = pd.read_json(path_or_buf='./webbizz_10_articles.jsonl', lines=True)

collection.add(
    documents=list(jsonObj.input),
    ids=list(jsonObj.result)
)

In [2]:
# We have a set of 20 questions, each asking about different business aspects. 
# There is a specific summary that is the target answer for the question, which is linked by UUID.
# We upload these questions into a scenario set, so that they can be used in a test run in the next cell
import os
from okareo import Okareo

OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

scenario = okareo.upload_scenario_set(file_path='./webbizz_retrieval_questions.jsonl', scenario_name="Retrieval Articles Scenario")
print(scenario) # use scenario set id in output for the next step

ScenarioSetResponse(scenario_id='20d5421c-7966-4a5c-96b2-b987af0eaf45', project_id='f7bfcff0-2b8d-4fec-ab20-51669cd3e732', time_created=datetime.datetime(2024, 1, 10, 2, 5, 48, 391559), type='SEED', tags=None, name='Retrieval Articles Scenario', seed_data=[], scenario_count=0, scenario_input=[], additional_properties={})


In [3]:
# Perform a test run using a scenario set loaded in the previous cell 
from datetime import datetime
from okareo_api_client.models import TestRunType
from okareo.model_under_test import CustomModel


# Callable to be applied to each scenario in the scenario set
def call_model(input: str):
    results = collection.query(
        query_texts=[input],
        n_results=5
    )
    parsed_ids_with_scores = []
    for i in range(0, len(results['distances'][0])):
        score = (2 - results['distances'][0][i]) / 2 # this turns cosine distance into a 0 to 1 cosine similarity score
        parsed_ids_with_scores.append((results['ids'][0][i], score))

    # return a tuple of (parsed_ids_with_scores, overall model response context)
    return parsed_ids_with_scores, {'model_data': input} 

# this will return a model if it already exists or create a new one if it doesn't
model_under_test = okareo.register_model(name="vectordb_retrieval_test", model=CustomModel(model_invoker=call_model, name="custom retrieval"))

test_run_name=f"Retrieval Test Run {datetime.now().strftime('%m-%d %H:%M:%S')}"

test_run_item = model_under_test.run_test(scenario=scenario, # use the scenario from the scenario set uploaded in the previous step
                                          name=test_run_name,
                                          test_run_type=TestRunType.INFORMATION_RETRIEVAL,
                                          calculate_metrics=True)

# display model level metrics for the test run
print(test_run_item.model_metrics.to_dict())
print(f"https://app.okareo.com/project/{test_run_item.project_id}/eval/{test_run_item.id}")

{'Accuracy@k': {'1': 0.8, '2': 0.9, '3': 1.0, '4': 1.0, '5': 1.0, '6': 1.0, '7': 1.0, '8': 1.0, '9': 1.0, '10': 1.0}, 'Precision@k': {'1': 0.8, '2': 0.45, '3': 0.33333333333333326, '4': 0.25, '5': 0.20000000000000004, '6': 0.16666666666666663, '7': 0.14285714285714285, '8': 0.125, '9': 0.11111111111111112, '10': 0.10000000000000002}, 'Recall@k': {'1': 0.8, '2': 0.9, '3': 1.0, '4': 1.0, '5': 1.0, '6': 1.0, '7': 1.0, '8': 1.0, '9': 1.0, '10': 1.0}, 'NDCG@k': {'1': 0.8, '2': 0.8630929753571458, '3': 0.9130929753571457, '4': 0.9130929753571457, '5': 0.9130929753571457, '6': 0.9130929753571457, '7': 0.9130929753571457, '8': 0.9130929753571457, '9': 0.9130929753571457, '10': 0.9130929753571457}, 'MRR@k': {'1': 0.8, '2': 0.85, '3': 0.8833333333333332, '4': 0.8833333333333332, '5': 0.8833333333333332, '6': 0.8833333333333332, '7': 0.8833333333333332, '8': 0.8833333333333332, '9': 0.8833333333333332, '10': 0.8833333333333332}, 'MAP@k': {'1': 0.8, '2': 0.85, '3': 0.8833333333333332, '4': 0.88333